## In London, if someone is looking to open a restaurant, where should it be?

### Description of problem:
A client is asking to recommend an area to open a luxury Asian restaurant in London. They want their restaurant to be in an area with higher asian population, that area's household income to adjust price, area access to public transport.

Interests:
1. Asian restaurants owner/entrepreneurs
2. Franchising asian restaurants

### Description of data:
#### 1. Demographics of different areas in London, top 3 area with highest asian population %

https://en.wikipedia.org/wiki/Demography_of_London

#### 2. Average household income in those area of London

https://data.london.gov.uk/download/earnings-place-residence-borough/1686ef1c-b169-442d-8877-e7e49788f668/earnings-residence-borough.xlsx

#### 3. Amount of asian restaurants in those area competition; their performance

Using foursquare

#### 4. Locations that can be chosen as potential restaurant locations; Locations access to main road

Gaps found on map after analysis, folium map; identifying clusters using K-means.


## Methodology
---
#### 1. Demographics of different areas in London, top 3 area with highest asian population %

In [1]:
#Scraping data from web, converting in dataframe
import pandas as pd
import numpy as np

demograph_dfs = pd.read_html('https://en.wikipedia.org/wiki/Demography_of_London')
demograph_df = demograph_dfs[4]
demograph_df.head()

,Local authority,White,Mixed,Asian,Black,Other
0,Barnet,64.1,4.8,18.5,7.7,4.8
1,Barking and Dagenham,58.3,4.2,15.9,20.0,1.6
2,Bexley,81.9,2.3,6.6,8.5,0.8
3,Brent,36.3,5.1,34.1,18.8,5.8
4,Bromley,84.3,3.5,5.2,6.0,0.9


In [2]:
#Dataframe cleaning
demograph_df.drop(['White','Mixed','Black','Other'], axis=1,inplace=True)


In [3]:
#Finding the top 3 boroughs with highest asian population %
demograph_df.rename(columns={'Local authority': 'Borough','Asian':'Asian'}).sort_values('Asian', ascending=False).head(3)

,Borough,Asian
24,Newham,43.5
13,Harrow,42.6
25,Redbridge,41.8


### The London boroughs with highest asian population % is Newham, Harrow, Redbridge
---
#### 2. Average  income in those area of London

In [4]:
#Extract data from xlsx
url_earn = 'https://data.london.gov.uk/download/earnings-place-residence-borough/1686ef1c-b169-442d-8877-e7e49788f668/earnings-residence-borough.xlsx'
earn_per_borough_dfs = pd.read_excel(url_earn, sheet_name=1)
earn_per_borough_dfs.head()

,Code,Area,2002,Unnamed: 3,2003,Unnamed: 5,2004,Unnamed: 7,2005,Unnamed: 9,...,2016,Unnamed: 31,2017,Unnamed: 33,2018,Unnamed: 35,2019,Unnamed: 37,2020,Unnamed: 39
0,NaN,NaN,Pay (£),conf %,Pay (£),conf %,Pay (£),conf %,Pay (£),conf %,...,Pay (£),conf %,Pay (£),conf %,Pay (£),conf %,Pay (£),conf %,Pay (£),conf %
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00AA,City of London,!,!,!,!,#,#,#,#,...,#,#,#,#,909.4,19,#,#,#,#
3,00AB,Barking and Dagenham,383.3,5.5,354.9,6.5,360.1,5.2,375.1,5.1,...,462.2,4.5,461,5.1,479.1,4.9,471,6.1,500.4,7.5
4,00AC,Barnet,427.4,5.1,450.1,5,453.3,5.6,442.3,5.3,...,485.6,5.4,522.6,4.5,536.6,4.6,536.4,4.4,520.2,5.5


In [5]:
#Looking at only 2020 results
epb2020_df = earn_per_borough_dfs[['Area',2020]]
epb2020_df.head()

,Area,2020
0,NaN,Pay (£)
1,NaN,NaN
2,City of London,#
3,Barking and Dagenham,500.4
4,Barnet,520.2


In [6]:
epb2020_df=epb2020_df.rename(columns={'Area': 'Borough', 2020:'2020'})
epb2020_df.loc[epb2020_df['Borough'].isin(['Newham', 'Redbridge','Harrow'])]

,Borough,2020
16,Harrow,566.7
26,Newham,519.4
27,Redbridge,581.4


### The London borough with highest asian population % and highest income is Redbridge.
---
#### 3. Amount of asian restaurants in those area competition

In [7]:
!pip install geopy
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

In [14]:
import requests 


In [72]:
address = 'Redbridge, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.5763203 0.0454097


In [73]:
# Foursquare API
import json, requests

CLIENT_ID = 'HLFO5MSFGHJQ3ZBNN1WHM1EAKLCH4RRLVBVWF5BYZNIIIJPV' # your Foursquare ID
CLIENT_SECRET = 'BBXCADEEI1QZAKXAYHDAHIPEPXE3PNCIZBCPDUAVALETFMU1' # your Foursquare Secret
ACCESS_TOKEN = 'PQWX3GWYCE1LHVFXSXJCJBRBBHJPVNNGT51OHASOTLLZTEX5' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
search_query = 'Asian cuisine'
radius = 1000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()
results

Your credentails:
CLIENT_ID: HLFO5MSFGHJQ3ZBNN1WHM1EAKLCH4RRLVBVWF5BYZNIIIJPV
CLIENT_SECRET:BBXCADEEI1QZAKXAYHDAHIPEPXE3PNCIZBCPDUAVALETFMU1
Asian cuisine .... OK!


{'meta': {'code': 200, 'requestId': '603a830550c1fe01f3a00bea'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': []}}

#### The dataframe above had showed that there was 0 asian restaurant in London Borough of Redbridge within radius 1000.

This could mean two hypothesis: 1) The area has no competition and is great for the opportunity. 2) The area is not suitable for a restaurant considering that there were 1 asian restaurants. Considering that this area has the highest asian population %, the amount of asian restaurants is lacking as well, therefore indicating that this might not be the best area for the restaurant.
### conclusion: Redbridge is not recommended for asian restaurant.

In [74]:
venues = results['response']['venues']
dataframe=json_normalize(venues)
dataframe

<ipython-input-74-ea808d34cbe2>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe=json_normalize(venues)


""


In [75]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

KeyError: "None of [Index(['name', 'categories', 'id'], dtype='object')] are in the [columns]"

In [54]:
#filtered anything without the word restaurant, cuisine

dataframe_catfil = dataframe_filtered[dataframe_filtered["categories"].str.contains('Restaurant', na=False) | dataframe_filtered["name"].str.contains('Restaurant', na=False) | dataframe_filtered["name"].str.contains('cuisine', na=False)]
dataframe_catfil


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
4,Bhangra Beat Indian Cuisine,Indian Restaurant,49 Chigwell Rd.,51.590403,0.034375,"[{'label': 'display', 'lat': 51.59040269539847...",1743,E18 1NG,GB,London,Greater London,United Kingdom,"[49 Chigwell Rd., London, Greater London, E18 ...",NaN,4ac518e1f964a52041aa20e3
5,Jaipur of Chigwell,Indian Restaurant,734 Chigwell Rd,51.606798,0.055907,"[{'label': 'display', 'lat': 51.6067984, 'lng'...",3469,IG8 8AL,GB,Woodford,Greater London,United Kingdom,"[734 Chigwell Rd, Woodford, Greater London, IG...",NaN,5638fe4b60b2753ece75a56a
6,Masterchef Chinese & Thai Cuisine,Thai Restaurant,NaN,51.606424,0.054434,"[{'label': 'display', 'lat': 51.606424, 'lng':...",3408,NaN,GB,NaN,NaN,United Kingdom,NaN,NaN,51f2ba00498e7425c8f0beac
8,Dera Punjabi Cuisine,Indian Restaurant,NaN,51.560587,-0.009796,"[{'label': 'display', 'lat': 51.560587, 'lng':...",4202,NaN,GB,NaN,NaN,United Kingdom,NaN,NaN,519141e2498eb313734fe2dd
11,Ochi Cuisine,Caribbean Restaurant,364 Lea Bridge Road,51.570217,-0.025167,"[{'label': 'display', 'lat': 51.570217, 'lng':...",4929,E10 7LD,GB,London,Greater London,United Kingdom,"[364 Lea Bridge Road, London, Greater London, ...",NaN,584db1b2ed0a3236a5b7dfd7
12,The Taste of Jamaica Caribbean Cuisine,Caribbean Restaurant,664 Green Lane,51.563083,0.115812,"[{'label': 'display', 'lat': 51.5630828, 'lng'...",5089,IG3 9RX,GB,Essex,Essex,United Kingdom,"[664 Green Lane, Essex, IG3 9RX]",NaN,58a734a2fd27166ea412cc3f
13,K's Cuisine Nigerian Restaurant,None,"664 Green Lane, Ilford IG3 9RX",51.563093,0.116370,"[{'label': 'display', 'lat': 51.5630934, 'lng'...",5125,IG3 9RX,GB,NaN,NaN,United Kingdom,"[664 Green Lane, Ilford IG3 9RX, IG3 9RX]",NaN,5638ffa560b2753ece75bb57


In [59]:

import folium

In [71]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)


folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Masterchef Chinese & Thai Cuisine',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


for lat, lng, label in zip(dataframe_catfil.lat, dataframe_catfil.lng, dataframe_catfil.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

---
#### Moving on to the second choice: Harrow

In [77]:
#Performing the same operation.
address = 'Harrow, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.596827149999996 -0.33731605402671094


In [85]:
# Foursquare API
import json, requests

CLIENT_ID = 'HLFO5MSFGHJQ3ZBNN1WHM1EAKLCH4RRLVBVWF5BYZNIIIJPV' # your Foursquare ID
CLIENT_SECRET = 'BBXCADEEI1QZAKXAYHDAHIPEPXE3PNCIZBCPDUAVALETFMU1' # your Foursquare Secret
ACCESS_TOKEN = 'PQWX3GWYCE1LHVFXSXJCJBRBBHJPVNNGT51OHASOTLLZTEX5' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
search_query = 'Asian restaurant'
radius = 1000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()
results

Your credentails:
CLIENT_ID: HLFO5MSFGHJQ3ZBNN1WHM1EAKLCH4RRLVBVWF5BYZNIIIJPV
CLIENT_SECRET:BBXCADEEI1QZAKXAYHDAHIPEPXE3PNCIZBCPDUAVALETFMU1
Asian restaurant .... OK!


{'meta': {'code': 200, 'requestId': '603a83b64253cd5b56f06697'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '522c5dc911d203fd8e7c5034',
    'name': 'Pan Asian Restaurant',
    'location': {'lat': 51.594969834728346,
     'lng': -0.32554104880276075,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.594969834728346,
       'lng': -0.32554104880276075}],
     'distance': 840,
     'cc': 'GB',
     'country': 'United Kingdom'},
    'categories': [{'id': '4bf58dd8d48988d142941735',
      'name': 'Asian Restaurant',
      'pluralName': 'Asian Restaurants',
      'shortName': 'Asian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1614447543',
    'hasPerk': False},
   {'id': '4e1df101fa7653a9b2e806f1',
    'name': 'Punjab Restaurant',
    'location': {'address': '27 Station Rd',
     'lat': 51.600531464011695,


In [86]:
venues = results['response']['venues']
dataframe=json_normalize(venues)
dataframe

<ipython-input-86-ea808d34cbe2>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe=json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.address,location.city,location.state,location.formattedAddress,location.postalCode,location.crossStreet,venuePage.id
0,522c5dc911d203fd8e7c5034,Pan Asian Restaurant,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1614447543,False,51.594970,-0.325541,"[{'label': 'display', 'lat': 51.59496983472834...",840,GB,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4e1df101fa7653a9b2e806f1,Punjab Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1614447543,False,51.600531,-0.332319,"[{'label': 'display', 'lat': 51.60053146401169...",537,GB,United Kingdom,27 Station Rd,Harrow on the Hill,Greater London,"[27 Station Rd, Harrow on the Hill, Greater Lo...",NaN,NaN,NaN
2,4cb4b65b75ebb60c5da2e4ad,Everest Restaurant and Bar,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1614447543,False,51.594078,-0.331908,"[{'label': 'display', 'lat': 51.59407750710088...",483,GB,United Kingdom,NaN,Edgware,NaN,[Edgware],NaN,NaN,NaN
3,4c496df09f2ad13ac98e4553,Koreana Restaurant,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",v-1614447543,False,51.590434,-0.334949,"[{'label': 'display', 'lat': 51.59043361097029...",730,GB,United Kingdom,32 Headstone Drive,Harrow on the Hill,Greater London,"[32 Headstone Drive, Harrow on the Hill, Great...",HA3 5QH,NaN,NaN
4,4bf0326151f2c9b60052f192,The Ruby Lounge,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1614447543,False,51.598125,-0.352550,"[{'label': 'display', 'lat': 51.598125, 'lng':...",1063,GB,United Kingdom,Broadwalk,Harrow on the Hill,Greater London,"[Broadwalk (Pinner Road), Harrow on the Hill, ...",HA 2 6,Pinner Road,NaN
5,562e5e5b498e1e1dafb18059,Chennai Srilalitha Veg Restaurant,"[{'id': '5413605de4b0ae91d18581a9', 'name': 'S...",v-1614447543,False,51.593775,-0.323904,"[{'label': 'display', 'lat': 51.59377536430263...",987,GB,United Kingdom,196 Kenton Road,Harrow,NaN,"[196 Kenton Road, Harrow, HA3 8BX]",HA3 8BX,NaN,145266877
6,56544c34498e2b8675f15b27,Saras Restaurant & Bar,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1614447543,False,51.583043,-0.335083,"[{'label': 'display', 'lat': 51.58304327584647...",1542,GB,United Kingdom,"28 The Broadwalk, Pinner Road, North Harrow",London,Greater London,"[28 The Broadwalk, Pinner Road, North Harrow, ...",HA2 6ED,NaN,NaN
7,4bdf02760d69b713230e66d3,Vintage Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1614447543,False,51.584882,-0.332240,"[{'label': 'display', 'lat': 51.584882, 'lng':...",1375,GB,United Kingdom,207 Station Road,Harrow,NaN,"[207 Station Road, Harrow, 2TP]",2TP,NaN,NaN


In [87]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,address,city,state,formattedAddress,postalCode,crossStreet,id
0,Pan Asian Restaurant,Asian Restaurant,51.594970,-0.325541,"[{'label': 'display', 'lat': 51.59496983472834...",840,GB,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,522c5dc911d203fd8e7c5034
1,Punjab Restaurant,Indian Restaurant,51.600531,-0.332319,"[{'label': 'display', 'lat': 51.60053146401169...",537,GB,United Kingdom,27 Station Rd,Harrow on the Hill,Greater London,"[27 Station Rd, Harrow on the Hill, Greater Lo...",NaN,NaN,4e1df101fa7653a9b2e806f1
2,Everest Restaurant and Bar,Restaurant,51.594078,-0.331908,"[{'label': 'display', 'lat': 51.59407750710088...",483,GB,United Kingdom,NaN,Edgware,NaN,[Edgware],NaN,NaN,4cb4b65b75ebb60c5da2e4ad
3,Koreana Restaurant,Korean Restaurant,51.590434,-0.334949,"[{'label': 'display', 'lat': 51.59043361097029...",730,GB,United Kingdom,32 Headstone Drive,Harrow on the Hill,Greater London,"[32 Headstone Drive, Harrow on the Hill, Great...",HA3 5QH,NaN,4c496df09f2ad13ac98e4553
4,The Ruby Lounge,Indian Restaurant,51.598125,-0.352550,"[{'label': 'display', 'lat': 51.598125, 'lng':...",1063,GB,United Kingdom,Broadwalk,Harrow on the Hill,Greater London,"[Broadwalk (Pinner Road), Harrow on the Hill, ...",HA 2 6,Pinner Road,4bf0326151f2c9b60052f192
5,Chennai Srilalitha Veg Restaurant,Sri Lankan Restaurant,51.593775,-0.323904,"[{'label': 'display', 'lat': 51.59377536430263...",987,GB,United Kingdom,196 Kenton Road,Harrow,NaN,"[196 Kenton Road, Harrow, HA3 8BX]",HA3 8BX,NaN,562e5e5b498e1e1dafb18059
6,Saras Restaurant & Bar,Indian Restaurant,51.583043,-0.335083,"[{'label': 'display', 'lat': 51.58304327584647...",1542,GB,United Kingdom,"28 The Broadwalk, Pinner Road, North Harrow",London,Greater London,"[28 The Broadwalk, Pinner Road, North Harrow, ...",HA2 6ED,NaN,56544c34498e2b8675f15b27
7,Vintage Restaurant,Indian Restaurant,51.584882,-0.332240,"[{'label': 'display', 'lat': 51.584882, 'lng':...",1375,GB,United Kingdom,207 Station Road,Harrow,NaN,"[207 Station Road, Harrow, 2TP]",2TP,NaN,4bdf02760d69b713230e66d3


In [88]:
#filtered anything without the word restaurant, cuisine

dataframe_catfil = dataframe_filtered[dataframe_filtered["categories"].str.contains('Restaurant', na=False) | dataframe_filtered["name"].str.contains('Restaurant', na=False) | dataframe_filtered["name"].str.contains('cuisine', na=False)]
dataframe_catfil


,name,categories,lat,lng,labeledLatLngs,distance,cc,country,address,city,state,formattedAddress,postalCode,crossStreet,id
0,Pan Asian Restaurant,Asian Restaurant,51.594970,-0.325541,"[{'label': 'display', 'lat': 51.59496983472834...",840,GB,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,522c5dc911d203fd8e7c5034
1,Punjab Restaurant,Indian Restaurant,51.600531,-0.332319,"[{'label': 'display', 'lat': 51.60053146401169...",537,GB,United Kingdom,27 Station Rd,Harrow on the Hill,Greater London,"[27 Station Rd, Harrow on the Hill, Greater Lo...",NaN,NaN,4e1df101fa7653a9b2e806f1
2,Everest Restaurant and Bar,Restaurant,51.594078,-0.331908,"[{'label': 'display', 'lat': 51.59407750710088...",483,GB,United Kingdom,NaN,Edgware,NaN,[Edgware],NaN,NaN,4cb4b65b75ebb60c5da2e4ad
3,Koreana Restaurant,Korean Restaurant,51.590434,-0.334949,"[{'label': 'display', 'lat': 51.59043361097029...",730,GB,United Kingdom,32 Headstone Drive,Harrow on the Hill,Greater London,"[32 Headstone Drive, Harrow on the Hill, Great...",HA3 5QH,NaN,4c496df09f2ad13ac98e4553
4,The Ruby Lounge,Indian Restaurant,51.598125,-0.352550,"[{'label': 'display', 'lat': 51.598125, 'lng':...",1063,GB,United Kingdom,Broadwalk,Harrow on the Hill,Greater London,"[Broadwalk (Pinner Road), Harrow on the Hill, ...",HA 2 6,Pinner Road,4bf0326151f2c9b60052f192
5,Chennai Srilalitha Veg Restaurant,Sri Lankan Restaurant,51.593775,-0.323904,"[{'label': 'display', 'lat': 51.59377536430263...",987,GB,United Kingdom,196 Kenton Road,Harrow,NaN,"[196 Kenton Road, Harrow, HA3 8BX]",HA3 8BX,NaN,562e5e5b498e1e1dafb18059
6,Saras Restaurant & Bar,Indian Restaurant,51.583043,-0.335083,"[{'label': 'display', 'lat': 51.58304327584647...",1542,GB,United Kingdom,"28 The Broadwalk, Pinner Road, North Harrow",London,Greater London,"[28 The Broadwalk, Pinner Road, North Harrow, ...",HA2 6ED,NaN,56544c34498e2b8675f15b27
7,Vintage Restaurant,Indian Restaurant,51.584882,-0.332240,"[{'label': 'display', 'lat': 51.584882, 'lng':...",1375,GB,United Kingdom,207 Station Road,Harrow,NaN,"[207 Station Road, Harrow, 2TP]",2TP,NaN,4bdf02760d69b713230e66d3


In [89]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Pan Asian Restaurant',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_catfil.lat, dataframe_catfil.lng, dataframe_catfil.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### The dataframe above had showed that there were several Asian restaurants in London Borough of Harrow, indicating that this could be a potential area for an Asian restaurant.

1) This area will have competition with 7 other restaurants. However, this area is still far from the center of London where traffic is a key component. This will not attract that many customers. 

### conclusion: Harrow can be considered for asian restaurant.
---
#### Moving on to the third choice: Newham

In [97]:
#Performing the same operation.
address = 'Newham, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.52999955 0.02931796029382208


In [91]:
CLIENT_ID = 'IEZSIHBKO4OURVSO3OJIMHCCAKSH23Q1CEPL1EFIAX1WSZYF' # your Foursquare ID
CLIENT_SECRET = 'TL1XDFPAOU1HKCCSDREW3WK340VXIHIQQREGVYI5VSBNTGPN' # your Foursquare Secret
ACCESS_TOKEN = 'T0KDP0WESDOSOQYOGBWP12CDC4XOMY5DZUXBBC1TRNIHBPVE' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
search_query = 'Asian restaurant'
radius = 1000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()
results

venues = results['response']['venues']
dataframe=json_normalize(venues)
dataframe

Your credentails:
CLIENT_ID: IEZSIHBKO4OURVSO3OJIMHCCAKSH23Q1CEPL1EFIAX1WSZYF
CLIENT_SECRET:TL1XDFPAOU1HKCCSDREW3WK340VXIHIQQREGVYI5VSBNTGPN
Asian restaurant .... OK!


<ipython-input-91-83b669dee375>:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe=json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.postalCode,location.city,location.state,location.crossStreet
0,505f56a1e4b0cf965a0a8eb5,Kabayan Cafe & Restaurant,"[{'id': '4eb1bd1c3b7b55596b4a748f', 'name': 'F...",v-1614447745,False,51.532243,0.036029,"[{'label': 'display', 'lat': 51.532243, 'lng':...",527,GB,United Kingdom,[United Kingdom],NaN,NaN,NaN,NaN,NaN
1,57ea3e4d498e7532a524915d,Bar & Restaurant Bela-Vista,"[{'id': '4def73e84765ae376e57713a', 'name': 'P...",v-1614447745,False,51.526296,0.030122,"[{'label': 'display', 'lat': 51.526296, 'lng':...",416,GB,United Kingdom,"[564 Barking Road, London, Greater London, E13...",564 Barking Road,E13 9JU,London,Greater London,NaN
2,4df7abc718a89a0e291c863e,Aasai Dosai Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1614447745,False,51.530068,0.038234,"[{'label': 'display', 'lat': 51.530068, 'lng':...",617,GB,United Kingdom,"[3 - 5 Barking Road, East Ham, Greater London,...",3 - 5 Barking Road,E9 1PW,East Ham,Greater London,NaN
3,4ec963d5be7ba4fc70b76bb4,Jolsagor restaurant,[],v-1614447745,False,51.534684,0.033435,"[{'label': 'display', 'lat': 51.53468418263492...",594,GB,United Kingdom,"[261 barking road, London, Greater London, Uni...",261 barking road,NaN,London,Greater London,NaN
4,4ec964965c5ce271bfdaae3a,Thai-thai restaurant,"[{'id': '4bf58dd8d48988d152941735', 'name': 'A...",v-1614447745,False,51.535871,0.029502,"[{'label': 'display', 'lat': 51.5358714083186,...",653,GB,United Kingdom,"[250 barking road, east ham, London, Greater L...","250 barking road, east ham",NaN,London,Greater London,NaN
5,4ac518def964a520c4a920e3,Stratford's Restaurant,"[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",v-1614447745,False,51.532562,0.018352,"[{'label': 'display', 'lat': 51.53256175478631...",811,GB,United Kingdom,"[7 Stratford Road, London, Greater London, W8 ...",7 Stratford Road,W8 6RF,London,Greater London,NaN
6,5debb389b431b1000877d801,Men's Cafe & Restaurant,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1614447745,False,51.532475,0.015021,"[{'label': 'display', 'lat': 51.532475, 'lng':...",1027,GB,United Kingdom,"[235 Plaistow Road, London, Greater London, E1...",235 Plaistow Road,E15 3EU,London,Greater London,NaN
7,4b9d4fbaf964a520aba236e3,Samrat,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1614447745,False,51.521297,0.020402,"[{'label': 'display', 'lat': 51.52129702405322...",1148,GB,United Kingdom,"[320-322 Barking Rd., London, Greater London, ...",320-322 Barking Rd.,E13 8HL,London,Greater London,NaN
8,59c29aa1c5b11c627ff79cbd,Himalaya Restaurant and Take Away,"[{'id': '52e81612bcbc57f1066b79f8', 'name': 'P...",v-1614447745,False,51.538575,0.033431,"[{'label': 'display', 'lat': 51.53857477456405...",996,GB,United Kingdom,"[9-10 Carlton terrace (Green st), Upton Park, ...",9-10 Carlton terrace,E7 8LH,Upton Park,Greater London,Green st
9,58d11b6353051117c845fd41,Restaurant,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1614447745,False,51.522710,0.035540,"[{'label': 'display', 'lat': 51.52271, 'lng': ...",918,GB,United Kingdom,"[London, Greater London, E13 8RU, United Kingdom]",NaN,E13 8RU,London,Greater London,NaN


In [98]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,postalCode,city,state,crossStreet,id
0,Kabayan Cafe & Restaurant,Filipino Restaurant,51.532243,0.036029,"[{'label': 'display', 'lat': 51.532243, 'lng':...",527,GB,United Kingdom,[United Kingdom],NaN,NaN,NaN,NaN,NaN,505f56a1e4b0cf965a0a8eb5
1,Bar & Restaurant Bela-Vista,Portuguese Restaurant,51.526296,0.030122,"[{'label': 'display', 'lat': 51.526296, 'lng':...",416,GB,United Kingdom,"[564 Barking Road, London, Greater London, E13...",564 Barking Road,E13 9JU,London,Greater London,NaN,57ea3e4d498e7532a524915d
2,Aasai Dosai Indian Restaurant,Indian Restaurant,51.530068,0.038234,"[{'label': 'display', 'lat': 51.530068, 'lng':...",617,GB,United Kingdom,"[3 - 5 Barking Road, East Ham, Greater London,...",3 - 5 Barking Road,E9 1PW,East Ham,Greater London,NaN,4df7abc718a89a0e291c863e
3,Jolsagor restaurant,None,51.534684,0.033435,"[{'label': 'display', 'lat': 51.53468418263492...",594,GB,United Kingdom,"[261 barking road, London, Greater London, Uni...",261 barking road,NaN,London,Greater London,NaN,4ec963d5be7ba4fc70b76bb4
4,Thai-thai restaurant,Arepa Restaurant,51.535871,0.029502,"[{'label': 'display', 'lat': 51.5358714083186,...",653,GB,United Kingdom,"[250 barking road, east ham, London, Greater L...","250 barking road, east ham",NaN,London,Greater London,NaN,4ec964965c5ce271bfdaae3a
5,Stratford's Restaurant,French Restaurant,51.532562,0.018352,"[{'label': 'display', 'lat': 51.53256175478631...",811,GB,United Kingdom,"[7 Stratford Road, London, Greater London, W8 ...",7 Stratford Road,W8 6RF,London,Greater London,NaN,4ac518def964a520c4a920e3
6,Men's Cafe & Restaurant,Breakfast Spot,51.532475,0.015021,"[{'label': 'display', 'lat': 51.532475, 'lng':...",1027,GB,United Kingdom,"[235 Plaistow Road, London, Greater London, E1...",235 Plaistow Road,E15 3EU,London,Greater London,NaN,5debb389b431b1000877d801
7,Samrat,Indian Restaurant,51.521297,0.020402,"[{'label': 'display', 'lat': 51.52129702405322...",1148,GB,United Kingdom,"[320-322 Barking Rd., London, Greater London, ...",320-322 Barking Rd.,E13 8HL,London,Greater London,NaN,4b9d4fbaf964a520aba236e3
8,Himalaya Restaurant and Take Away,Pakistani Restaurant,51.538575,0.033431,"[{'label': 'display', 'lat': 51.53857477456405...",996,GB,United Kingdom,"[9-10 Carlton terrace (Green st), Upton Park, ...",9-10 Carlton terrace,E7 8LH,Upton Park,Greater London,Green st,59c29aa1c5b11c627ff79cbd
9,Restaurant,Cafeteria,51.522710,0.035540,"[{'label': 'display', 'lat': 51.52271, 'lng': ...",918,GB,United Kingdom,"[London, Greater London, E13 8RU, United Kingdom]",NaN,E13 8RU,London,Greater London,NaN,58d11b6353051117c845fd41


In [99]:
#filtered anything without the word restaurant, cuisine

dataframe_catfil = dataframe_filtered[dataframe_filtered["categories"].str.contains('Restaurant', na=False) | dataframe_filtered["name"].str.contains('Restaurant', na=False) | dataframe_filtered["name"].str.contains('cuisine', na=False)]
dataframe_catfil


,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,postalCode,city,state,crossStreet,id
0,Kabayan Cafe & Restaurant,Filipino Restaurant,51.532243,0.036029,"[{'label': 'display', 'lat': 51.532243, 'lng':...",527,GB,United Kingdom,[United Kingdom],NaN,NaN,NaN,NaN,NaN,505f56a1e4b0cf965a0a8eb5
1,Bar & Restaurant Bela-Vista,Portuguese Restaurant,51.526296,0.030122,"[{'label': 'display', 'lat': 51.526296, 'lng':...",416,GB,United Kingdom,"[564 Barking Road, London, Greater London, E13...",564 Barking Road,E13 9JU,London,Greater London,NaN,57ea3e4d498e7532a524915d
2,Aasai Dosai Indian Restaurant,Indian Restaurant,51.530068,0.038234,"[{'label': 'display', 'lat': 51.530068, 'lng':...",617,GB,United Kingdom,"[3 - 5 Barking Road, East Ham, Greater London,...",3 - 5 Barking Road,E9 1PW,East Ham,Greater London,NaN,4df7abc718a89a0e291c863e
4,Thai-thai restaurant,Arepa Restaurant,51.535871,0.029502,"[{'label': 'display', 'lat': 51.5358714083186,...",653,GB,United Kingdom,"[250 barking road, east ham, London, Greater L...","250 barking road, east ham",NaN,London,Greater London,NaN,4ec964965c5ce271bfdaae3a
5,Stratford's Restaurant,French Restaurant,51.532562,0.018352,"[{'label': 'display', 'lat': 51.53256175478631...",811,GB,United Kingdom,"[7 Stratford Road, London, Greater London, W8 ...",7 Stratford Road,W8 6RF,London,Greater London,NaN,4ac518def964a520c4a920e3
6,Men's Cafe & Restaurant,Breakfast Spot,51.532475,0.015021,"[{'label': 'display', 'lat': 51.532475, 'lng':...",1027,GB,United Kingdom,"[235 Plaistow Road, London, Greater London, E1...",235 Plaistow Road,E15 3EU,London,Greater London,NaN,5debb389b431b1000877d801
7,Samrat,Indian Restaurant,51.521297,0.020402,"[{'label': 'display', 'lat': 51.52129702405322...",1148,GB,United Kingdom,"[320-322 Barking Rd., London, Greater London, ...",320-322 Barking Rd.,E13 8HL,London,Greater London,NaN,4b9d4fbaf964a520aba236e3
8,Himalaya Restaurant and Take Away,Pakistani Restaurant,51.538575,0.033431,"[{'label': 'display', 'lat': 51.53857477456405...",996,GB,United Kingdom,"[9-10 Carlton terrace (Green st), Upton Park, ...",9-10 Carlton terrace,E7 8LH,Upton Park,Greater London,Green st,59c29aa1c5b11c627ff79cbd
9,Restaurant,Cafeteria,51.522710,0.035540,"[{'label': 'display', 'lat': 51.52271, 'lng': ...",918,GB,United Kingdom,"[London, Greater London, E13 8RU, United Kingdom]",NaN,E13 8RU,London,Greater London,NaN,58d11b6353051117c845fd41
10,Taris African Restaurant,African Restaurant,51.521676,0.021096,"[{'label': 'display', 'lat': 51.52167596277876...",1087,GB,United Kingdom,"[East Ham, Greater London, United Kingdom]",NaN,NaN,East Ham,Greater London,NaN,4ec012fd5c5c3d470cdacc40


In [101]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Newham',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_catfil.lat, dataframe_catfil.lng, dataframe_catfil.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### The dataframe above had showed that there were several Asian restaurants in London Borough of Newham, indicating that this could be a potential area for an Asian restaurant.

1) This area will have competition with 10 other restaurants. This area is closer to the city center of London and will be a achieve a higher traffic. However, this area is the least profitable as the average income in this area is the lowest of 3.

### conclusion: 
---
#### Extra: Looking at the busiest London Area: Covent Garden

In [102]:
#Performing the same operation.
address = 'Covent Garden, London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

CLIENT_ID = 'IEZSIHBKO4OURVSO3OJIMHCCAKSH23Q1CEPL1EFIAX1WSZYF' # your Foursquare ID
CLIENT_SECRET = 'TL1XDFPAOU1HKCCSDREW3WK340VXIHIQQREGVYI5VSBNTGPN' # your Foursquare Secret
ACCESS_TOKEN = 'T0KDP0WESDOSOQYOGBWP12CDC4XOMY5DZUXBBC1TRNIHBPVE' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
search_query = 'Asian restaurant'
radius = 1000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()
results

venues = results['response']['venues']
dataframe=json_normalize(venues)

filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#filtered anything without the word restaurant, cuisine

dataframe_catfil = dataframe_filtered[dataframe_filtered["categories"].str.contains('Restaurant', na=False) | dataframe_filtered["name"].str.contains('Restaurant', na=False) | dataframe_filtered["name"].str.contains('cuisine', na=False)]
dataframe_catfil


venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup=address,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_catfil.lat, dataframe_catfil.lng, dataframe_catfil.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Your credentails:
CLIENT_ID: IEZSIHBKO4OURVSO3OJIMHCCAKSH23Q1CEPL1EFIAX1WSZYF
CLIENT_SECRET:TL1XDFPAOU1HKCCSDREW3WK340VXIHIQQREGVYI5VSBNTGPN
Asian restaurant .... OK!


<ipython-input-102-f666893e4576>:26: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe=json_normalize(venues)


#### Stiff competition between 20+ asian restaurants. However higher traffic but at the same time higher costs of rent.
---
### Conclusion: London Borough of Harrow is the best area for opening an Asian restaurants with least competition in a high asian population area with moderate average income. However, the location will exhibit less traffic as it is far from London busy city center. The second choice will be London Borough of Newham which is by far closest to the London city center amongst the 3 areas. However, client will need to reconsider if the restaurant is best for luxurious customers because Newham has the lowest average amongst the three. Redbridge is not recommended for an Asian restaurants as the search did not show a confident amount of restaurants, indicating that other asian restaurants chose not to open in Redbridge.

### Extra studies on London's busiest area, covent garden has shown 20+ asian restaurants. The competition will be stiff and the running cost is estimated to be high as well. However, it will attract the luxury customers. More analysis will have to be done.


Future work:
The investigation neglected a lot of factors that might affect the decision of opening a restaurant in London, for example: pedestrian traffic, type of area(i.e. residential or commercial or business), running costs(i.e. rent in higher traffic areas). Therefore, in order to get a better grasp of whether it is a good decision to open an asian restaurant, more specialized data is needed.

One big factor is that this analysis is formed on the basis of Foursquare API accurately providing a full picture of the area. There is possibility that the API did not show all the restaurants available in the area.